# Read Data

In [1]:
import numpy as np
import pandas as pd
import os

data_dict = {}
skip = [".DS_Store", "displayModel.m", "displayTrial.m", "MANUAL.txt", "README.txt"]
files = os.listdir("HMP_Dataset/")
for file_name in files:
    if file_name not in skip:
        path = "HMP_Dataset/" + file_name
        files = os.listdir(path)
        data = []
        index = 0
        for name in files:
            tmp = pd.read_csv(path + "/" + name, sep=" ", header=None).values
            data = np.vstack([data, tmp]) if len(data) else tmp
            data_dict[file_name + "/" + str(index)] = data
            index += 1

# Vector quantization 

In [2]:
import math

# Cut and flatten signals 
def preprocessing_data_dict(data_dict):
    preprocessed_data_dict = {}
    sample_size = 32
    for key in data_dict:
        size = len(data_dict[key])
        x = 0.5 # overlap parameter
        cut = [data_dict[key][i:i+sample_size] for i in range(0, size, int(sample_size*x))]
        filtered = [x for x in cut if len(x) == sample_size]
        preprocessed_data_dict[key] = np.array([np.array([j for i in e for j in i]) for e in filtered])
    return preprocessed_data_dict

preprocessed_data_dict = preprocessing_data_dict(data_dict) 


In [3]:
data = []
for key in preprocessed_data_dict:
    data = np.vstack([data, np.array(preprocessed_data_dict[key])]) if len(data) else np.array(preprocessed_data_dict[key])
    

In [ ]:
from sklearn.cluster import KMeans


y = []
X = []
label_dict = {}
label_mapping = ["Brush_teeth", "Climb_stairs", "Comb_hair", "Descend_stairs", "Drink_glass", "Eat_meat", "Eat_soup", "Getup_bed", "Liedown_bed", "Pour_water", "Sitdown_chair", "Standup_chair", "Use_telephone", "Walk"]
i = 0

print "Starting clustering"
kmeans = KMeans(n_clusters=480).fit(data)
print "End of clustering"
for key in preprocessed_data_dict:
    if i % 10 == 0:
        print i
    k = re.sub(r'[^a-zA-Z_]+', '', key)
    tmp = kmeans.predict(preprocessed_data_dict[key]) # transform to get a freq vector from each file
    x = np.zeros(480)
    for v in tmp:
        x[v] += 1 
    label_dict[k] = np.vstack([label_dict[k], np.array(x)]) if len(label_dict[k]) else np.array(x)
    X.append(x)
    y.append(label_mapping.index(k)) 


Starting clustering


# Histogram representation

In [ ]:
print label_dict.labels()

In [ ]:
%matplotlib inline 
import matplotlib.pyplot as plt

#vals = np.fromiter(vq_bylabel.itervalues(), dtype=float)
#print vals
for key in label_dict:
   # print np.mean(vq_bylabel[key], axis=0)
    plt.hist(np.mean(label_dict[key], axis=0), bins=40, alpha=0.7, rwidth=0.90, density=True)
    plt.xlabel('Cluster labels')
    plt.ylabel('Frequency')
    plt.title("Histogram for " + key)
    plt.xticks(range(0, 481, 40))
    plt.show()

# Classification with random forest

In [ ]:
from sklearn.model_selection import KFold
from sklearn.ensemble import RandomForestClassifier

kfolds = KFold(n_splits=3)
avg_error_rate = 0
for train_index, test_index in kfolds.split(X):
    X_train, X_test = X[train_index], X[test_index]
    y_train, y_test = y[train_index], y[test_index]
    clf = RandomForestClassifier(n_estimators=100)
    clf.fit(X_train, y_train)
    error_rate = (clf.predict(X_test) != y_test).sum() / float(len(y_test))
    avg_error_rate += error_rate
avg_error_rate /= 3
print avg_error_rate